In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
from sklearn import datasets

In [12]:
from sklearn.tree import DecisionTreeClassifier

In [19]:
from sklearn.model_selection import train_test_split

In [54]:
from sklearn.neighbors import KNeighborsClassifier

In [8]:
def standard_scale(x):
    res = (x - x.mean(axis=0)) / x.std(axis=0)
    return res

In [47]:
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

**1. Обучить любую модель классификации на датасете IRIS до применения самописного PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.**

In [6]:
iris = datasets.load_iris()
X = iris.data

In [15]:
y = iris.target

In [7]:
X = X.astype(float)

In [9]:
X = standard_scale(X)

In [25]:
# Найдем собственные векторы и собственные значения
covariance_matrix = X.T @ X

In [26]:
eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

In [28]:
# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

In [29]:
# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

In [30]:
print('Собственные значения и собственные векторы в порядке убывания:')
for i in eig_pairs:
    print(i)

Собственные значения и собственные векторы в порядке убывания:
(437.77467247979916, array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654]))
(137.10457072021052, array([-0.37741762, -0.92329566, -0.02449161, -0.06694199]))
(22.01353133569726, array([-0.71956635,  0.24438178,  0.14212637,  0.63427274]))
(3.107225464292839, array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))


In [31]:
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперь оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[72.96244541329989, 22.850761786701764, 3.6689218892828777, 0.5178709107154734]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


In [32]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack([eig_pairs[i][1].reshape(4,1) for i in range(2)])

In [33]:
# Сформируем новую матрицу "объекты-признаки"
X_pca = X.dot(W)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=10, shuffle=True)

In [39]:
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.33, random_state=11, shuffle=True)

In [40]:
model_1 = DecisionTreeClassifier()

In [41]:
model_1.fit(X_train, y_train)

DecisionTreeClassifier()

In [42]:
model_pca = DecisionTreeClassifier()

In [43]:
model_pca.fit(X_train_pca, y_train_pca)

DecisionTreeClassifier()

In [49]:
y_pred = model_1.predict(X_test)

In [48]:
y_pred_pca = model_pca.predict(X_test_pca)

In [51]:
accuracy(y_pred, y_test)

0.9

In [52]:
accuracy(y_pred_pca, y_test_pca)

0.84

In [55]:
model_2 = KNeighborsClassifier()

In [59]:
model_2.fit(X_train, y_train)

KNeighborsClassifier()

In [63]:
y_pred_2 = model_2.predict(X_test)

In [64]:
accuracy(y_pred_2, y_test)

0.94

In [65]:
model_2_pca = KNeighborsClassifier()

In [66]:
model_2_pca.fit(X_train_pca, y_train_pca)

KNeighborsClassifier()

In [68]:
y_pred_2_pca = model_2_pca.predict(X_test_pca)

In [69]:
accuracy(y_pred_2_pca, y_test_pca)

0.88

**2. *Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции [numpy.linalg.svd()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html)**

In [135]:
from numpy import diag

In [334]:
def pca_svd(X, n):
    print(X.shape)
    # SVD декомпозиция матрицы X
    u, s, vt = np.linalg.svd(X)#, full_matrices=False)
    # вывод на печать элементов:
    print(u.shape, s.shape, vh.shape)
    # обрезаем результаты разложения матрицы по количеству необходимых измерений
    # sigma = zeros((x.shape[0], x.shape[1])) 
    #sh[:X.shape[0], :X.shape[0]] = diag(s) #returns error. need transposing?
    #sh = sh[:, :n]    
    uh = u[:,:n]    
    vth = vt[:n, :]
    # вывод на печать формы уменьшенных компонент
    #print(uh.shape, sh.shape, vth.shape)
    print(uh.shape, vth.shape)
    
    #сравнение с начальным массивом
    #x_init = uh @ np.diag(sh) @ vth 
    #print(x_init==x)
    
    #преобразование матрицы
    #T = u.dot(sh)
    X_svd = X.dot(vth.T)
    
    return X_svd

Проводим вычисление

In [335]:
X_svd = pca_svd(X, 2)


(150, 4)
(150, 150) (4,) (4, 4)
(150, 2) (2, 4)


In [306]:
X_svd.shape

(150, 2)

In [308]:
X_svd

array([[-2.26470281, -0.4800266 ],
       [-2.08096115,  0.67413356],
       [-2.36422905,  0.34190802],
       [-2.29938422,  0.59739451],
       [-2.38984217, -0.64683538],
       [-2.07563095, -1.48917752],
       [-2.44402884, -0.0476442 ],
       [-2.23284716, -0.22314807],
       [-2.33464048,  1.11532768],
       [-2.18432817,  0.46901356],
       [-2.1663101 , -1.04369065],
       [-2.32613087, -0.13307834],
       [-2.2184509 ,  0.72867617],
       [-2.6331007 ,  0.96150673],
       [-2.1987406 , -1.86005711],
       [-2.26221453, -2.68628449],
       [-2.2075877 , -1.48360936],
       [-2.19034951, -0.48883832],
       [-1.898572  , -1.40501879],
       [-2.34336905, -1.12784938],
       [-1.914323  , -0.40885571],
       [-2.20701284, -0.92412143],
       [-2.7743447 , -0.45834367],
       [-1.81866953, -0.08555853],
       [-2.22716331, -0.13725446],
       [-1.95184633,  0.62561859],
       [-2.05115137, -0.24216355],
       [-2.16857717, -0.52714953],
       [-2.13956345,

Проверка результата при помощи стандартной функции sklearn.

In [317]:
from numpy import array
from sklearn.decomposition import TruncatedSVD

In [318]:
svd = TruncatedSVD(n_components=2)

In [322]:
svd.fit(X)

TruncatedSVD()

In [324]:
result = svd.transform(X)
print(result)

[[-2.26470281  0.4800266 ]
 [-2.08096115 -0.67413356]
 [-2.36422905 -0.34190802]
 [-2.29938422 -0.59739451]
 [-2.38984217  0.64683538]
 [-2.07563095  1.48917752]
 [-2.44402884  0.0476442 ]
 [-2.23284716  0.22314807]
 [-2.33464048 -1.11532768]
 [-2.18432817 -0.46901356]
 [-2.1663101   1.04369065]
 [-2.32613087  0.13307834]
 [-2.2184509  -0.72867617]
 [-2.6331007  -0.96150673]
 [-2.1987406   1.86005711]
 [-2.26221453  2.68628449]
 [-2.2075877   1.48360936]
 [-2.19034951  0.48883832]
 [-1.898572    1.40501879]
 [-2.34336905  1.12784938]
 [-1.914323    0.40885571]
 [-2.20701284  0.92412143]
 [-2.7743447   0.45834367]
 [-1.81866953  0.08555853]
 [-2.22716331  0.13725446]
 [-1.95184633 -0.62561859]
 [-2.05115137  0.24216355]
 [-2.16857717  0.52714953]
 [-2.13956345  0.31321781]
 [-2.26526149 -0.3377319 ]
 [-2.14012214 -0.50454069]
 [-1.83159477  0.42369507]
 [-2.61494794  1.79357586]
 [-2.44617739  2.15072788]
 [-2.10997488 -0.46020184]
 [-2.2078089  -0.2061074 ]
 [-2.04514621  0.66155811]
 

Данные достаточно близки между собой, хотя и присутствуют определенные расхождения.